## Deploy Mphasis Optimize.AI Expert Identifier Model Package from AWS Marketplace 


This sample notebook shows you how to deploy Expert Identifier is machine learning based model that uses information present in any incident/ticket management data such as: Ticket ID, Ticket Solver Id, Ticket Priority, Ticket Category, Ticket Submission and Resolved date and identifies the right expert to be assigned to a specific ticket or incident request. It can optimise ticket allocation, decreases the ticket resolution time and improve KPIs (Key Performance Indicators) such as customer satisfaction, adherence to SLA (Service Level Agreement), MTTR (Mean Time to Resolve), cost to company, etc.
 using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to  Mphasis Optimize.AI Expert Identifier. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page  Mphasis Optimize.AI Expert Identifier.
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [6]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/marketplace-expert'

In [7]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import urllib.request
import numpy as np
import pandas as pd

In [8]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [9]:
model_name='expert'

content_type='text/csv'

real_time_inference_instance_type='ml.t2.medium'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [ ]:

def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [11]:
sample_input = "Input_file.csv"

<Add code snippet that shows the payload contents>

In [12]:
input_df = pd.read_csv(sample_input)
input_df.head()

,Request ID,Request Resolved By,Request Submitted Date and Time,Request Priority,Request Resolved Date and Time,Request Category,Assigned/Unassigned,Request Status
0,INC_1,Solver_1,4/1/2016 4:04,3-Low,4/2/2016 10:16,Request_Category_1,Assigned,Closed
1,INC_2,Solver_2,4/1/2016 4:52,3-Low,4/1/2016 4:52,request_Category_2,Assigned,Closed
2,INC_3,Solver_3,4/1/2016 4:23,3-Low,4/1/2016 4:23,request_Category_2,Assigned,Closed
3,INC_4,Solver_4,4/1/2016 4:10,3-Low,4/1/2016 4:31,Request_Category_3,Assigned,Closed
4,INC_5,Solver_5,4/1/2016 5:27,3-Low,4/1/2016 5:31,Request_Category_4,Assigned,Closed


In [13]:
file_name = sample_input
output_file_name = "sample_output"

#### C. Perform real-time inference

In [14]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    $output_file_name

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [15]:
out_df = pd.read_csv(output_file_name)
out_df.head()

,Unnamed: 0,Request ID,Request Submitted Date and Time,Request Priority,Request Category,Request Status,Request Resolved By
0,0,INC_10502,2016-04-01 14:05:00,3-Low,Request_Category_14,Open,Solver_72
1,1,INC_10501,2016-04-01 14:07:00,3-Low,Request_Category_243,Open,Solver_75
2,2,INC_10510,2016-04-01 14:10:00,3-Low,Request_Category_5,Open,Solver_18
3,3,INC_10508,2016-04-01 14:11:00,3-Low,Request_Category_168,Open,Solver_32
4,4,INC_10484,2016-04-01 14:13:00,3-Low,Request_Category_5,Open,Solver_106


#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [16]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
#upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

In [ ]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
#output is available on following path
transformer.output_path

### 4. Clean-up

#### A. Delete the model

In [17]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

